In [1]:
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/AmericanPics/Traces'
all_files = glob.glob(os.path.join(path, "*"))
print("Total files : " + str(len(all_files)))
x = 0
while x < len(all_files):
  k= all_files[x].rindex('/')
  tmp = all_files[x][k+1:]
  print(str(x) + " represents " + tmp)
  x = x + 1

Total files : 9
0 represents rsrch_0.csv_output
1 represents mds_0.csv_output
2 represents ts_0.csv_output
3 represents stg_1.csv_output
4 represents hm_0.csv_output
5 represents stg_0.csv_output
6 represents web_0.csv_output
7 represents usr_0.csv_output
8 represents proj_0.csv_output


In [4]:
f = all_files[1]
print("Working with file" + str(f))
cols = ['IO_num','Timestamp','LBA','Deathtime_RWI','Size']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
print(len(df))

Working with file/content/drive/My Drive/AmericanPics/Traces/mds_0.csv_output
1035178


In [5]:
lba_list = df['LBA'].tolist()
lba_max = max(lba_list)
high = len(bin(lba_max))
print(str(lba_max) + " is highest with " + str(high) + " bits" )

lba_min = min(lba_list)
low = len(bin(lba_min))
print(str(lba_min) + " is lowest with " + str(low) + " bits" )


36415303680 is highest with 38 bits
16384 is lowest with 17 bits


In [6]:
from collections import Counter
common_list_dict = Counter(lba_list).most_common(1000)

common_list = []
for x in common_list_dict:
    common_list.append(x[0])
print(len(common_list))


coverage = 0
for x in common_list_dict:
    if x[0] in common_list:
        coverage = coverage + x[1]
        
print("Coverage for top " + str(len(common_list)) +" LBA is " + str((coverage/len(df))*100) +" %")

1000
Coverage for top 1000 LBA is 87.06550950657761 %


In [7]:
lba_low_list = []
lba_high_list = []
lba_class_list = []
lba_delta_list = []

prime_num = 97

slice_index = int(high/2)

count = 0
while (count <len(lba_list)- 1):
    tmp = bin(lba_list[count])
    
    lba_high = tmp[:slice_index]
    lba_high_dec = int(lba_high,2)
    lba_high_list.append(lba_high_dec% prime_num)
    
    lba_low =  tmp[-(slice_index):]
    lba_low_dec = int(lba_low,2)
    lba_low_list.append(lba_low_dec% prime_num)
    
    lba_delta_list.append(abs(lba_list[count+1] - lba_list[count]))
    x = lba_list[count]
    if x not in common_list:
        lba_class_list.append(-1)
    else:
        lba_class_list.append(x)
    
    if(count+1 == len(lba_list) - 1):
        break

    count = count+1
    

print(len(df))
print(len(lba_list))
print(len(lba_high_list))
print(len(lba_low_list))
print(len(lba_class_list))
print(len(lba_delta_list))

1035178
1035178
1035177
1035177
1035177
1035177


In [8]:
# Place all lba_high, lab_low,lba_class and lba_delta in the df and drop unnecessary columns
df = df[:-1] #drop bottom 
df['LBA_high'] = lba_high_list
df['LBA_low'] = lba_low_list
df['LBA_class'] = lba_class_list
df['LBA_delta'] = lba_delta_list

In [9]:
common_list_dict = Counter(lba_delta_list).most_common(1000)

common_list = []
for x in common_list_dict:
    common_list.append(x[0])
print(len(common_list))


coverage = 0
for x in common_list_dict:
    if x[0] in common_list:
        coverage = coverage + x[1]
        
print("Coverage for top " + str(len(common_list)) +" LBA delta is " + str((coverage/len(df))*100) +" %")

1000
Coverage for top 1000 LBA delta is 71.0646585076755 %


In [10]:
# Normalize TS_Delta and RWI
import pandas as pd
from sklearn import preprocessing


x = df[['LBA_delta']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['LBA_delta_norm'] = pd.DataFrame(x_scaled)

x = df[['LBA']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['LBA_norm'] = pd.DataFrame(x_scaled)


x = df[['Deathtime_RWI']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['RWI_norm'] = pd.DataFrame(x_scaled)


In [11]:
import math

size_list = df['Size'].tolist()
death_rwi_list = df['Deathtime_RWI'].tolist()
size_class_list = []

count = -1
while (count < len(size_list) - 1):
    count = count + 1
    tmp_size = size_list[count]
    size_class_list.append(int(math.log2(tmp_size)))

print(len(df))
print(len(size_class_list))

df['Size_class'] = size_class_list


1035177
1035177


In [12]:
df.dtypes


IO_num              int64
Timestamp           int64
LBA                 int64
Deathtime_RWI       int64
Size                int64
LBA_high            int64
LBA_low             int64
LBA_class           int64
LBA_delta           int64
LBA_delta_norm    float64
LBA_norm          float64
RWI_norm          float64
Size_class          int64
dtype: object

In [13]:
# Sort by Timestamp and Drop all unnecessary cols 
# Dropping ThreadID since we created a duplicate class with numerical data
print("Before drop: {}".format(df.columns))


df.drop(['IO_num','Timestamp','LBA','Deathtime_RWI','Size','LBA_class','LBA_delta'], axis=1, inplace=True)
print("After drop: {}".format(df.columns))


Before drop: Index(['IO_num', 'Timestamp', 'LBA', 'Deathtime_RWI', 'Size', 'LBA_high',
       'LBA_low', 'LBA_class', 'LBA_delta', 'LBA_delta_norm', 'LBA_norm',
       'RWI_norm', 'Size_class'],
      dtype='object')
After drop: Index(['LBA_high', 'LBA_low', 'LBA_delta_norm', 'LBA_norm', 'RWI_norm',
       'Size_class'],
      dtype='object')


In [14]:
# Sort by Timestamp and Drop all unnecessary cols 
# Dropping ThreadID since we created a duplicate class with numerical data
print("Before drop: {}".format(df.columns))


df.drop(['LBA_high','LBA_low'], axis=1, inplace=True)
print("After drop: {}".format(df.columns))


Before drop: Index(['LBA_high', 'LBA_low', 'LBA_delta_norm', 'LBA_norm', 'RWI_norm',
       'Size_class'],
      dtype='object')
After drop: Index(['LBA_delta_norm', 'LBA_norm', 'RWI_norm', 'Size_class'], dtype='object')


In [17]:
df.dtypes

LBA_delta_norm    float64
LBA_norm          float64
RWI_norm          float64
Size_class          int64
dtype: object

In [15]:
import math

# Finding the value 75th percentile of TimeStamp
training_pt = math.floor((len(df)*0.80)) 
print(training_pt)


df_train = df[:training_pt]
df_test = df[training_pt+1:]
print("Number of observations in train = {} \
    \n Number of observations in test = {}".format(df_train.shape[0],
                                                  df_test.shape[0]))

print("Training set has {} observations.".format(len(df_train)))
print("Test set has {} observations.".format(len(df_test)))

828141
Number of observations in train = 828141     
 Number of observations in test = 207035
Training set has 828141 observations.
Test set has 207035 observations.


In [16]:
import numpy as np

def to_sequences(seq_size, obs):
    length_obs = obs.shape[0]
    #print("length_obs")
    #print(length_obs)
    x = []
    y = []

    for i in range(length_obs-SEQUENCE_SIZE-1):
#         window = obs[i:(i+SEQUENCE_SIZE)]
        window = obs.iloc[i:(i+SEQUENCE_SIZE)]
#         after_window = obs[i+SEQUENCE_SIZE]
        after_window = obs.iloc[i+SEQUENCE_SIZE]
#         window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(np.array(window))
#         y.append(after_window)
        y.append(after_window['RWI_norm'])
        
#     return np.array(x),np.array(y)
    return np.array(x),np.array(y)
    
    
SEQUENCE_SIZE = 32
x_train,y_train = to_sequences(SEQUENCE_SIZE,df_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,df_test)

print("Training : Shape of X: {} & Y shape = {}".format(x_train.shape,len(y_train)))
print("Test : Shape of X: {} & Y shape = {} ".format(x_test.shape,len(y_test)))

KeyboardInterrupt: ignored

In [59]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(828096, 32, 4)
(206976, 32, 4)
(828096,)
(206976,)


In [73]:
x_test = x_test[:-976]
y_test = y_test[:-976]

In [79]:
look_back = 32
dims = len(df.dtypes)
batch_size = 10

model = Sequential()
model.add(LSTM(80, batch_input_shape=(batch_size, look_back, dims), stateful=True, return_sequences=True))
model.add(LSTM(80, batch_input_shape=(batch_size, look_back, dims),stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(100):
  model.fit(x_train, y_train, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
  model.reset_states()
# make predictions

62292/82800 [=====================>........] - ETA: 8:00 - loss: 0.0050

KeyboardInterrupt: ignored

In [ ]:
trainPredict = model.predict(x_train, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(y_train, batch_size=batch_size)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()